# Games RSs

In [2]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
from scipy.sparse import random
from scipy.sparse import save_npz

In [4]:
from scipy.sparse import load_npz
from scipy.sparse import vstack

In [5]:
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "lab_data/Video_Games_5.json"
N = 10

## Анализ данных

In [6]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
        
    return pd.DataFrame(rows, columns=cols)

In [7]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


In [8]:
print("min-max количество объектов на пользователя:", 
      df.groupby("uid").iid.nunique().min(), df.groupby("uid").iid.nunique().max())
print("min-max количество пользователей на объект:", 
      df.groupby("iid").uid.nunique().min(), df.groupby("iid").uid.nunique().max())

min-max количество объектов на пользователя: 5 773
min-max количество пользователей на объект: 5 802


In [9]:
# проверяем, есть ли случаи, когда один и тот же пользователь оставляет отзывы на один и тот же объект
df.groupby(["uid", "iid"]).review.count().unique()  # ура, таких случаев нет

array([1], dtype=int64)

In [10]:
print("Количество объектов:", df.iid.unique().size)
print("Количество пользователей:", df.uid.unique().size)

Количество объектов: 10672
Количество пользователей: 24303


## Готовим выборки

In [11]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [12]:
training_df, test_df = split_df_by_dt(df, p=0.8)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


In [13]:
def clean_df(df, min_review_per_uid, min_review_per_iid):
    """Функция удаляет из df строки, соответствующие пользователям и объектам, 
    у которых меньше min_review_per_uid и min_review_per_iid отзывов соответственно
    """
    _df = df.copy()
    while True:
        review_per_uid = _df.groupby("uid").review.count()
        bad_uids = review_per_uid[review_per_uid < min_review_per_uid].index
    
        review_per_iid = _df.groupby("iid").review.count()
        bad_iids = review_per_iid[review_per_iid < min_review_per_iid].index
        
        if bad_uids.shape[0] > 0 or bad_iids.shape[0] > 0:
            _df = _df[(~_df.uid.isin(bad_uids)) & (~_df.iid.isin(bad_iids))]
        else:
            break
    return _df

 ## Метрика

In [14]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [15]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## Разбиение задачи на блоки

In [16]:
Nsplit = 25
uid_to_ind_train = {}
uid_to_ind_test = {}

unique_users_train = training_df.uid.unique()
n_unique_train = len(unique_users_train)

unique_users_test = test_df.uid.unique()
n_unique_test = len(unique_users_test)

for uid in unique_users_train:
    uid_to_ind_train[uid] = uid_to_ind_train.setdefault(uid, len(uid_to_ind_train))
    
for uid in unique_users_test:
    uid_to_ind_test[uid] = uid_to_ind_test.setdefault(uid, len(uid_to_ind_test))
    
ind_to_uid_train = { v: k for k, v in uid_to_ind_train.items() }
ind_to_uid_test = { v: k for k, v in uid_to_ind_test.items() }

In [17]:
from math import ceil
block_len = ceil(n_unique_train/Nsplit)

# создаем список диапазонов индексов
block_list = [] # элементы списка - локальные индексы пользователей в блоке [ {id: uid, ...}, ...]
for i in range(Nsplit):
    start = i*block_len
    end = (i+1)*block_len
    loc_ind = 0
    dummy_dict = {}
    for j in range(start, min(end, n_unique_train)):
        dummy_dict[loc_ind] = ind_to_uid_train[j]
        loc_ind += 1
    block_list.append(dummy_dict)

In [18]:
 # элементы списка - локальные индексы пользователей в блоке [ {uid: id, ...}, ...]
uid_block_list = [ { v: k for k, v in block_list[i].items() } for i in range(Nsplit)] 

In [19]:
def get_block_number(id_, list_of_blocks):
    for i in range(len(list_of_blocks)):
        if id_ in list_of_blocks[i].keys():
            return i

In [20]:
data = []
for uid in unique_users_test:
    data.append((uid, get_block_number(uid, uid_block_list)))
uids_test_map = pd.DataFrame(data, columns=['uid', 'block'], dtype='int64')
del data

## Базовые классы для рекомендательной системы

In [21]:
class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}
    
class NonPersRecommender(BasicRecommender):
    
    def __init__(self, df):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df)
        
    def _prepare_recs(self, df):
        return pd.Series([])
    
    def get_recs(self, uid, top):
        from collections import OrderedDict
        return OrderedDict(self.recs[:top])
    
    def get_batch_recs(self, uids, top):
        non_pers_recs = self.get_recs(None, top)
        return {uid: non_pers_recs for uid in uids}

In [22]:
# выбирает соответсвие id объекта - rating, уже приобретенные объекты (exclude keys) исключаются
def select_item(source_matrix, keys_to_index, key_, exclude_keys):
    if key_ in exclude_keys:
        return 0
    else:
        return source_matrix[0, keys_to_index[key_]]

In [23]:
class ContentBasedRecommender_dummy(NonPersRecommender):
    
    def __init__(self, uid_to_ind, iid_to_ind, train_df, sim_matrix):
        super(NonPersRecommender, self).__init__()
        self.uid_to_ind = uid_to_ind
        self.iid_to_ind = iid_to_ind
        self.train_df = train_df
        self.sim_matrix = sim_matrix
        #self.recs = self._prepare_recs(uid_to_ind, iid_to_ind, train_df, sim_matrix)
        
    def get_recs(self, uids_to_recommend, top_k=10):
        self.recs = self._prepare_recs(uids_to_recommend, self.uid_to_ind, self.iid_to_ind, self.train_df, self.sim_matrix, top_k)
        return self.recs
        
    def _prepare_recs(self, uids_to_recommend, uid_to_ind, iid_to_ind, train_df, sim_matrix, top_k):
        out = {}
        for uid in uids_to_recommend:
            already_bought_items = train_df[train_df.uid == uid].iid.values
            sim_row = sim_matrix.getrow(uid_to_ind[uid]).todense()
            item_with_score = { iid: select_item(sim_row, iid_to_ind, iid, already_bought_items) for iid in iid_to_ind.keys() }
            dummy = pd.Series(item_with_score).sort_values(ascending=False)[:top_k].to_dict()
            out[uid] = dummy
        return out

## Векторизуем объекты

In [24]:
item_ids = training_df.iid.unique() # уникальные идентификаторы объектов в тренировочной выборке
id_to_ind = {}
texts = []

# для каждого id из item_ids формируется "суммарное" описание из отзывово нескольких пользователей
for id in item_ids:
    text = training_df[training_df.iid == id].review.str.cat(sep=' ')
    texts.append(text)
    id_to_ind[id] = id_to_ind.setdefault(id, len(id_to_ind)) 

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
my_tfidf = TfidfVectorizer(stop_words='english', max_features=60000)
items_matrix = my_tfidf.fit_transform(texts)

In [27]:
del texts

In [28]:
del my_tfidf

In [29]:
items_matrix 

<10098x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 7188097 stored elements in Compressed Sparse Row format>

In [30]:
items_matrix.nnz/(items_matrix .shape[0]*items_matrix .shape[1])

0.011863895490856275

In [31]:
DICT_LEN = items_matrix.shape[1]

## Строим рекомендации

In [32]:
from scipy.sparse import vstack
from sklearn.metrics.pairwise import cosine_similarity

gl_recs = {} #будущий словарь с рекомендациями

for i in range(Nsplit):
    block = block_list[i]
    uid_to_ind_loc = uid_block_list[i]                # локальное соответсвие uid - номер строки
    uids_to_recommend = uids_test_map[uids_test_map.block == i].uid.values
    for j in range(len(block)):
        uid = block[j]
        user_df = training_df[training_df.uid == uid] # выбираем кусок df, соответствующий пользователю
        user_items = user_df.iid                      # список объектов, оцененных пользователем
        user_time = user_df.dt                        # время отзывов пользователя
        user_vect = csr_matrix((1, DICT_LEN))         # инициализируем пустой вектор пользователя
        user_time = user_time/max(user_time)

        for iid, time in zip(user_items, user_time):
            item_vect = items_matrix.getrow(id_to_ind[iid])
            user_vect += item_vect.multiply(time)     # идея в том, что чем новее отзыв, тем большую актуальность тема представляет для него сейчас
            
        try:
            users_matrix = vstack([users_matrix, user_vect])
        except NameError:
            users_matrix = user_vect                 # составили "локальную" матрицу пользователей
            
    sim_matrix = cosine_similarity(users_matrix, items_matrix, dense_output=False) # матрица схожести
    recommender = ContentBasedRecommender_dummy(uid_to_ind_loc, id_to_ind, training_df, sim_matrix)
    recs = recommender.get_recs(uids_to_recommend)
    gl_recs.update(recs)
    del users_matrix, sim_matrix
    print("{} block done".format(i))

0 block done
1 block done
2 block done
3 block done
4 block done
5 block done
6 block done
7 block done
8 block done
9 block done
10 block done
11 block done
12 block done
13 block done
14 block done
15 block done
16 block done
17 block done
18 block done
19 block done
20 block done
21 block done
22 block done
23 block done
24 block done


In [32]:
# uids_to_recommend = uids_test_map[uids_test_map.block == i].uid.values
# test__ = test_df.loc[test_df.uid.isin(uids_to_recommend)]
# dummy_test_dict = get_test_dict(test__)

In [33]:
hit_ratio(gl_recs, test_dict)

0.06001467351430668

#### `HR@10` для item-based CF модели, созданной автором блокнота: 0.085

### Подcказки
* Определитесь с тем, что вы пытаетесь предсказать (рейтинги, вероятности, ...)
* Оптимальный способ вычисления матрицы схожести выглядит так:
 * Привести строки в матрице `user x item` к единичной длине (выделяет основные предпочтения пользователя)
 * Построить матрицу схожести `item x item`
 * Для каждого объекта оставить только $K$ наиболее похожих объектов
 * Для каждого объекта привести к единичной длине вектор схожести этого объекта (выделяет наиболее схожие объекты)
* Удалили ли вы из рекомендаций объекты, которые пользователь уже оценивал?
* Статья "Item-Based Top-N Recommendation Algorithms", Mukund Deshpande и George Karypis